In [1]:
import os, math, time
import numpy as np
import pandas as pd
import soundfile as sf
from dataclasses import dataclass
from typing import Dict, List, Optional, Union

import torch
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm

from transformers import (
    Wav2Vec2Config,
    Wav2Vec2FeatureExtractor,
    Wav2Vec2ForPreTraining,
)

from transformers.models.wav2vec2.modeling_wav2vec2 import (
    _compute_mask_indices,
    _sample_negative_indices,
)

# ---------------- CONFIG ----------------
CFG = {
    "train_csvs": ["/home/ubuntu/georgian-asr/train.csv"],
    "val_csvs": ["/home/ubuntu/georgian-asr/val.csv"],
    "sep": ",",                     # change if needed
    "audio_col": "path",
    "dur_col": "duration_sec",      # your column name
    "sr": 16000,

    "min_dur": 2.0,
    "max_dur": 20.0,

    "model_name": "facebook/wav2vec2-base",
    "load_pretrained": True,        # start from pretrained weights

    "batch_train": 16,
    "batch_val": 16,
    "num_workers": 8,

    "epochs": 20,
    "grad_accum": 2,
    "lr": 2e-5,
    "weight_decay": 0.01,
    "warmup_steps": 1000,

    "max_grad_norm": 1.0,
    "log_every_updates": 50,        # log every N optimizer updates
    "save_every_updates": 500,
    "out_dir": "wav2vec2-pretraining-jupyter",

    "mixed_precision": "bf16",      # "fp16" or "bf16" or "no"
}

os.makedirs(CFG["out_dir"], exist_ok=True)
device = "cuda" if torch.cuda.is_available() else "cpu"
device


/home/ubuntu/georgian-asr/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'cuda'

In [2]:
class CustomDataset(Dataset):
    def __init__(self, files, sep, sr, audio_column_name, duration_column_name, min_duration, max_duration):
        self.sep = sep
        self.sr = sr
        self.min_duration = min_duration
        self.max_duration = max_duration
        self.audio_column_name = audio_column_name
        self.duration_column_name = duration_column_name
        self.data = self.load_ds(files)

    def load_ds(self, all_files):
        li = []
        for filename in all_files:
            df = pd.read_csv(filename, sep=self.sep, engine="python")
            li.append(df)
        data = pd.concat(li, axis=0, ignore_index=True)

        if self.duration_column_name in data.columns:
            data = data[data[self.duration_column_name] >= self.min_duration]
            print("Mean duration:", float(data[self.duration_column_name].mean()))
        return data.reset_index(drop=True)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        wav = sf.read(item[self.audio_column_name])[0]

        # crop to max_dur (random window)
        if len(wav) / self.sr > self.max_duration:
            max_len = int(self.max_duration * self.sr)
            start = np.random.randint(0, len(wav) - max_len)
            wav = wav[start : start + max_len]

        return {"input_values": wav}


In [5]:
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(CFG["model_name"])
if not feature_extractor.do_normalize:
    raise ValueError("FeatureExtractor must have do_normalize=True for wav2vec2 pretraining")

config = Wav2Vec2Config.from_pretrained(CFG["model_name"])

if CFG["load_pretrained"]:
    model = Wav2Vec2ForPreTraining.from_pretrained(CFG["model_name"])
else:
    model = Wav2Vec2ForPreTraining(config)

model.to(device)


Wav2Vec2ForPreTraining(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (

In [6]:
@dataclass
class DataCollatorForWav2Vec2Pretraining:
    model: Wav2Vec2ForPreTraining
    feature_extractor: Wav2Vec2FeatureExtractor
    padding: Union[bool, str] = "longest"
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        batch = self.feature_extractor.pad(
            features,
            padding=self.padding,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch_size = batch["input_values"].shape[0]
        mask_len = int(self.model._get_feat_extract_output_lengths(batch["input_values"].shape[-1]))

        if batch.get("attention_mask") is not None:
            batch["sub_attention_mask"] = self.model._get_feature_vector_attention_mask(mask_len, batch["attention_mask"])

        features_shape = (batch_size, mask_len)

        mask_time_indices = _compute_mask_indices(
            features_shape,
            self.model.config.mask_time_prob,
            self.model.config.mask_time_length,
            attention_mask=batch.get("sub_attention_mask"),
        )
        sampled_negative_indices = _sample_negative_indices(
            features_shape,
            self.model.config.num_negatives,
            mask_time_indices=mask_time_indices,
        )

        # keep these on CPU too
        batch["mask_time_indices"] = torch.tensor(mask_time_indices, dtype=torch.long)
        batch["sampled_negative_indices"] = torch.tensor(sampled_negative_indices, dtype=torch.long)

        return batch


data_collator = DataCollatorForWav2Vec2Pretraining(model=model, feature_extractor=feature_extractor)


In [9]:
train_dataset = CustomDataset(
    CFG["train_csvs"], CFG["sep"], CFG["sr"], CFG["audio_col"], CFG["dur_col"],
    CFG["min_dur"], CFG["max_dur"]
)
val_dataset = CustomDataset(
    CFG["val_csvs"], CFG["sep"], CFG["sr"], CFG["audio_col"], CFG["dur_col"],
    CFG["min_dur"], CFG["max_dur"]
)

train_loader = DataLoader(
    train_dataset,
    batch_size=CFG["batch_train"],
    shuffle=True,
    collate_fn=data_collator,
    num_workers=CFG["num_workers"],
    pin_memory=True,
)

val_loader = DataLoader(
    val_dataset,
    batch_size=CFG["batch_val"],
    shuffle=False,
    collate_fn=data_collator,
    num_workers=CFG["num_workers"],
    pin_memory=True,
)

len(train_dataset), len(val_dataset)


Mean duration: 9.028636171653982
Mean duration: 8.854285431119921


(100388, 1009)

In [7]:
optimizer = torch.optim.AdamW(model.parameters(), lr=CFG["lr"], weight_decay=CFG["weight_decay"])

def lr_lambda(current_step):
    # linear warmup, then constant (simple + stable)
    if current_step < CFG["warmup_steps"]:
        return float(current_step) / float(max(1, CFG["warmup_steps"]))
    return 1.0

scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

use_fp16 = (CFG["mixed_precision"] == "fp16" and device == "cuda")
use_bf16 = (CFG["mixed_precision"] == "bf16" and device == "cuda")

scaler = torch.cuda.amp.GradScaler(enabled=use_bf16)


/tmp/ipykernel_2920/424004303.py:14: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_bf16)


In [10]:
model.train()
b = next(iter(train_loader))

# move tensors to GPU (only here, not in collator)
b = {k: (v.to(device) if torch.is_tensor(v) else v) for k, v in b.items()}

with torch.cuda.amp.autocast(
    enabled=(use_fp16 or use_bf16),
    dtype=(torch.float16 if use_fp16 else torch.bfloat16),
):
    out = model(**{k: v for k, v in b.items() if k != "sub_attention_mask"})


/tmp/ipykernel_2920/3242293628.py:7: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(


In [11]:
# ✅ Cell 8 — Training loop (single GPU) with safe device moves + fp16 support

from pathlib import Path
from torch.utils.tensorboard import SummaryWriter
import os

writer = SummaryWriter(os.path.join(CFG["out_dir"], "tb"))


def to_device(batch, device):
    return {k: (v.to(device, non_blocking=True) if torch.is_tensor(v) else v) for k, v in batch.items()}

def save_checkpoint(update_step: int):
    save_path = Path(CFG["out_dir"]) / f"checkpoint-{update_step}"
    save_path.mkdir(parents=True, exist_ok=True)
    model.save_pretrained(str(save_path))
    feature_extractor.save_pretrained(str(save_path))

@torch.no_grad()
def evaluate():
    model.eval()
    total_loss = 0.0
    total_cl = 0.0
    total_dl = 0.0
    total_masks = 0.0

    for batch in val_loader:
        batch = to_device(batch, device)
        batch.pop("sub_attention_mask", None)  # not needed for forward

        out = model(**batch)
        num_masks = batch["mask_time_indices"].sum().item()

        total_loss += out.loss.item()
        total_cl += out.contrastive_loss.item()
        total_dl += out.diversity_loss.item()
        total_masks += num_masks

    denom = max(1.0, total_masks)
    return {
        "val_loss_per_mask": total_loss / denom,
        "val_contrastive_per_mask": total_cl / denom,
        "val_diversity_per_mask": total_dl / denom,
    }

# ------------------- TRAIN -------------------
global_update = 0
model.train()

use_fp16 = (CFG["mixed_precision"] == "fp16" and device == "cuda")
use_bf16 = (CFG["mixed_precision"] == "bf16" and device == "cuda")

scaler = torch.cuda.amp.GradScaler(enabled=use_fp16)

for epoch in range(CFG["epochs"]):
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{CFG['epochs']}")
    optimizer.zero_grad(set_to_none=True)

    for step, batch in enumerate(pbar):
        # ✅ MUST move batch to GPU BEFORE forward
        batch = to_device(batch, device)
        batch.pop("sub_attention_mask", None)

        num_losses = batch["mask_time_indices"].sum().clamp(min=1)

        with torch.cuda.amp.autocast(
            enabled=(use_fp16 or use_bf16),
            dtype=(torch.float16 if use_fp16 else torch.bfloat16),
        ):
            out = model(**batch)

            # normalize by num masked positions + grad accumulation
            loss = (out.loss / num_losses) / CFG["grad_accum"]

        # backward
        if use_fp16:
            scaler.scale(loss).backward()
        else:
            loss.backward()

        # optimizer step (every grad_accum steps)
        if (step + 1) % CFG["grad_accum"] == 0:
            # clip
            if CFG["max_grad_norm"] is not None and CFG["max_grad_norm"] > 0:
                if use_fp16:
                    scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), CFG["max_grad_norm"])

            # step
            if use_fp16:
                scaler.step(optimizer)
                scaler.update()
            else:
                optimizer.step()

            optimizer.zero_grad(set_to_none=True)
            scheduler.step()
            global_update += 1

            # logs (every N optimizer updates)
            if global_update % CFG["log_every_updates"] == 0:
                with torch.no_grad():
                    loss_pm = (out.loss / num_losses).detach().float().cpu().item()
                    cl_pm   = (out.contrastive_loss / num_losses).detach().float().cpu().item()
                    dl_pm   = (out.diversity_loss / num_losses).detach().float().cpu().item()
                    lr      = float(scheduler.get_last_lr()[0])

                # ✅ print lines
                print(f"UPDATE {global_update} | loss={loss_pm:.4f} | cl={cl_pm:.4f} | dl={dl_pm:.4f} | lr={lr:.2e}")

                # ✅ tensorboard events
                writer.add_scalar("train/loss_per_mask", loss_pm, global_update)
                writer.add_scalar("train/contrastive_per_mask", cl_pm, global_update)
                writer.add_scalar("train/diversity_per_mask", dl_pm, global_update)
                writer.add_scalar("train/lr", lr, global_update)

            # save
            if global_update % CFG["save_every_updates"] == 0:
                save_checkpoint(global_update)

    # epoch eval
    metrics = evaluate()
    print(f"\nEpoch {epoch+1} eval:", metrics)
    model.train()

print("✅ training finished")


/tmp/ipykernel_2920/3823238765.py:53: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_fp16)
Epoch 1/20:   0%|          | 0/6275 [00:00<?, ?it/s]/tmp/ipykernel_2920/3823238765.py:66: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(
Epoch 1/20:   2%|▏         | 100/6275 [01:00<1:04:49,  1.59it/s]

UPDATE 50 | loss=3.4338 | cl=3.3399 | dl=0.9390 | lr=1.00e-06


Epoch 1/20:   3%|▎         | 200/6275 [02:00<1:03:03,  1.61it/s]

UPDATE 100 | loss=3.7232 | cl=3.6262 | dl=0.9705 | lr=2.00e-06


Epoch 1/20:   5%|▍         | 300/6275 [02:59<1:04:22,  1.55it/s]

...

UPDATE 62650 | loss=2.9985 | cl=2.9478 | dl=0.5064 | lr=2.00e-05


Epoch 20/20:  99%|█████████▊| 6194/6275 [1:00:47<00:51,  1.57it/s]

UPDATE 62700 | loss=3.3766 | cl=3.3263 | dl=0.5033 | lr=2.00e-05


Epoch 20/20: 100%|██████████| 6275/6275 [1:01:34<00:00,  1.70it/s]



Epoch 20 eval: {'val_loss_per_mask': 0.9583496668656803, 'val_contrastive_per_mask': 0.866832294989829, 'val_diversity_per_mask': 0.9151736798967861}
✅ training finished
